# Notebook 4 (Lecture 2) - Building a Multi‑Agent System

In this notebook we evolve from the single‑agent concept to a **coordinated ecosystem of specialized agents**. We combine two distinct domains – calendar management and desk reservation – and orchestrate them through a *Central Agent* that routes user requests.

> Goal: present a reusable pattern to integrate multiple ReAct agents, each with its own tools and prompt, keeping separated memories yet coherent coordination.

## Motivation
As functional scope grows, a single, monolithic prompt becomes fragile and hard to maintain. Splitting by competency:
- Improves precision (each agent has narrow context + domain policies)
- Enables independent evolution of capabilities (add a new domain without rewriting everything)
- Reduces *prompt bloat* and instruction interference
- Enables *selective reasoning* (only some agents use costlier reasoning models)

## Logical Architecture
```
User ─▶ Central Agent (Routing Decision) ─┬─▶ Calendar Agent (event tools)
                                         └─▶ Desk Agent (reservation tools)
```
Each domain agent:
- Has its own **system prompt** (policy, style, output format)
- Exposes semantically coherent tools (e.g. `add_event`, `reserve_desk`)
- Maintains **separate conversation memory** via a dedicated thread id

The Central Agent:
1. Interprets user intent (calendar vs desk vs ambiguous)
2. Routes via wrapper tools (`chat_with_calendar_agent`, `chat_with_desk_agent`)
3. Asks clarifying questions before routing if ambiguous

## What You Will Learn
| Topic | Why it matters |
|-------|----------------|
| Modular prompts | Limit interference & simplify maintenance |
| Intent-based routing | Prevent out-of-domain answers |
| Tool specialization | Shrink error surface & hallucinations |
| Per-agent memory | Cleaner local context, less noise |


## What You'll Build
You will implement:
1. Calendar Agent and tools to interact with the calendar.
2. Desk Agent and tools to interact with the desk management system. 
3. Central Agent (router that delegates rather than solves)

## Ready to Get Started?
Let’s set up the environment.

## 0. Setup the environment

In [1]:
from uuid import uuid4
from dotenv import load_dotenv

load_dotenv(override=True)

True

## 1. Calendar Agent

The Calendar Agent manages lightweight CRUD operations on events (list, add, delete).

### 1. Tool Definition

We define the operational tool set for the Calendar Agent. Each tool is atomic and free of policy; strategy (when/why) lives in the prompt.

#### Tools Provided
1. `get_current_date` – temporal anchor for relative expressions (today, tomorrow, day after tomorrow).
2. `convert_weekday_to_date` – robust mapping weekday → next ISO date (contextual to "today").
3. `list_events` – retrieve events for a date; also used for pre‑conflict checks.
4. `add_event` – events insertion 
5. `delete_event` – events removal

In [2]:
from tools.calendar import get_current_date, convert_weekday_to_date, list_events, add_event, delete_event

calendar_tools = [
    get_current_date,
    convert_weekday_to_date,
    list_events,
    add_event,
    delete_event
]

### 2. Initializing the LLM

We pick a lightweight chat model with low temperature for determinism. A larger or reasoning model can be swapped later for complex multi‑event reconciliation; here we optimize for responsiveness.

In [3]:
from langchain.chat_models import init_chat_model

CALENDAR_MODEL_NAME = "openai:gpt-4o-mini-2024-07-18"

calendar_llm = init_chat_model(CALENDAR_MODEL_NAME, temperature=0)

### 3. Crafting the Agent System Prompt

The Calendar Agent prompt encapsulates:
- Role + objective (event management, Italian output)
- Formatting rules (ISO internal, tabular output for lists)
- Safety policies (confirmation before delete)
- Disambiguation strategy (ask only for missing elements)
- Few-shot examples to anchor tool usage

This reduces cognitive overhead and nudges the model toward correct tool sequencing without verbose reasoning leakage.

In [4]:
CALENDAR_AGENT_PROMPT = """
# 📌 System Prompt — *Calendar Agent (gpt-4o mini)*

## Ruolo & Obiettivo

Sei un assistente calendario affidabile e conciso in lingua italiana. Aiuti l’utente a **vedere, aggiungere e rimuovere** impegni, interpretando richieste in linguaggio naturale e usando **solo** i tool disponibili. Non rivelare dettagli interni né schemi dei tool; mostra all’utente risultati chiari e conferme sintetiche.

## Principi di comportamento

* **Chiarezza prima di tutto.** Se la richiesta è ambigua (data/ora mancante, titoli multipli uguali), fai **una** domanda di chiarimento mirata prima di agire.
* **Niente catene di pensiero.** Non esporre ragionamenti passo-passo; mostra solo il risultato o le domande necessarie.
* **Localizzazione.** Rispondi in italiano, usa **orario 24h** e giorno della settimana in minuscolo.
* **Fuso orario di default:** Europe/Rome.
* **Formati coerenti:**

  * Interno/tool: **ISO 8601** `YYYY-MM-DD`; orario `HH:MM`.
  * Uscita utente: `ddd DD/MM/YYYY` (es. `ven 03/10/2025`) e `HH:MM`.
* **Conferme e toni:**

  * Conferma sempre dopo un’azione (aggiunta o eliminazione).
  * Per azioni distruttive (delete) **chiedi conferma** esplicita prima di procedere.

## Policy di uso dei tool

Usa esclusivamente i seguenti tool e solo quando servono allo scopo:

* `get_current_date` — per ancorare “oggi”, “domani”, “dopodomani”.
* `convert_weekday_to_date` — per espressioni come “lunedì”, “venerdì prossimo”.
* `list_events` — per elenchi o verifiche di conflitto.
* `add_event` — per creare un evento (richiede **data**, **ora**, **titolo**).
* `delete_event` — per rimuovere un evento per **titolo** in una **data**.

> Regola d’oro: prima di *add* o *delete*, verifica ed esplicita il **target** (data/ora/titolo) in modo non ambiguo; se manca un elemento necessario, chiedilo.

## Disambiguazione date/ore (procedura)

1. Se la richiesta contiene un **giorno della settimana** (es. “martedì”):
   → chiama `convert_weekday_to_date` per ottenere la prossima data utile.
2. Se contiene **oggi/domani/dopodomani**:
   → chiama `get_current_date` per ricavare la data di oggi e calcola quella relativa (senza esporre il calcolo).
3. Se la data è già in formato naturale (es. “3 ottobre 2025”):
   → normalizza internamente a `YYYY-MM-DD` prima di chiamare i tool.
4. Se **manca l’ora** per `add_event`: chiedi “A che ora?” (24h).
5. Se **manca il titolo**: chiedi “Come vuoi chiamare l’evento?”.

## Gestione conflitti e feedback

* Prima di `add_event`, se utile, esegui `list_events` sulla **stessa data** per rilevare collisioni orarie e informare l’utente.
* Se `add_event` segnala che l’evento esiste già in quell’ora, comunica il problema e proponi: “Vuoi cancellarlo e ricrearlo con un altro titolo/orario?”
* `list_events` senza risultati: riferisci il messaggio in tono neutro (“Nessun evento…”).
* Ordina sempre gli eventi per orario quando li mostri.

## Sicurezza per azioni distruttive

Prima di `delete_event`, **ripeti** target e chiedi conferma **sì/no** in una riga:

> “Confermi l’eliminazione di **‘titolo ’** del **YYYY-MM-DD**? (sì/no)”

Procedi solo se l’utente risponde “sì” (accetta anche “ok”, “confermo”).

## Stile di risposta

* Breve, pratico, niente gergo tecnico.
* Per gli elenchi, usa una tabella semplice:

| Ora   | Titolo       |
| ----- | ------------ |
| 09:30 | Dentista     |
| 14:00 | Allineamento |

* Conferme d’azione:

  * Aggiunta: `✅ Aggiunto “titolo” — ddd DD/MM/YYYY alle HH:MM.`
  * Eliminazione: `🗑️ Eliminato “titolo” — ddd DD/MM/YYYY.`
* Errori: spiega in una frase cosa è andato storto e come risolvere.

## Esempi d’uso (few-shot)

**Esempio 1 — Aggiungere con giorno della settimana**
**Utente:** «Aggiungi *Dentista* venerdì alle 09:30.»

1. `get_current_date` → determina oggi.
2. `convert_weekday_to_date`(“friday”) → ottieni la data ISO.
3. `add_event` con data/ora/titolo.
   **Risposta:** `✅ Aggiunto “Dentista” — ven DD/MM/YYYY alle 09:30.`

**Esempio 2 — Elencare eventi di una data**
**Utente:** «Cosa ho il 2025-10-03?»

1. `list_events` per quella data.
   **Risposta (con tabella o messaggio “nessun evento”).**

**Esempio 3 — Eliminare con conferma**
**Utente:** «Elimina *Dentista* di lunedì.»

1. `convert_weekday_to_date`(“monday”).
2. Chiedi conferma: “Confermi l’eliminazione di ‘Dentista’ del YYYY-MM-DD? (sì/no)”
3. Se sì → `delete_event`.
   **Risposta:** `🗑️ Eliminato “Dentista” — lun DD/MM/YYYY.`

**Esempio 4 — Dati mancanti**
**Utente:** «Aggiungi riunione domani.»

1. `get_current_date` → calcola domani.
2. Chiedi: “A che ora?” e “Titolo completo?” se necessario.

## Cose da non fare

* Non inventare dati mancanti (titolo, ora, data).
* Non mostrare né descrivere gli schemi dei tool o percorsi file.
* Non eseguire cancellazioni senza conferma.
* Non rivelare il ragionamento interno o piani step-by-step.


SESSION START

Greet the user only once at the beginning of a new thread. The user name is {user_name}. Current date is {current_date}.
"""

### 4. Dynamic Prompt

We can define a wrapper function for the system prompt to add user-related information or other details that may be useful. 

In [5]:
from langchain_core.runnables import RunnableConfig
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from tools.utils import get_current_date as get_current_date_util

def custom_prompt_calendar(state: dict, config: RunnableConfig):
    template = CALENDAR_AGENT_PROMPT
    prompt_template = PromptTemplate(
        template=template,
        input_variables=[
            "user_name", 
            "current_date"
        ],
    )

    user_name = config['configurable']['user']
    current_date = get_current_date_util()

    system_msg = SystemMessage(
        content=prompt_template.format(
            user_name=user_name, 
            current_date=current_date
        )
    )

    return [system_msg] + state["messages"]


### 5. Building the Calendar Agent

We compose the agent with:
- `model` already bound to tools (prevents hallucinated tool names)
- Dynamic `prompt` (inject user + current date)
- `MemorySaver` for per-thread continuity

Best practice: unique thread_id per agent to isolate context.

In [6]:
from langgraph.prebuilt import create_react_agent

from langgraph.checkpoint.memory import MemorySaver

calendar_agent_memory = MemorySaver()

calendar_agent = create_react_agent(
    model=calendar_llm.bind_tools(calendar_tools, parallel_tool_calls=False),
    tools=calendar_tools,
    prompt=custom_prompt_calendar,
    checkpointer=calendar_agent_memory,
)

calendar_agent_config = {
    "configurable": {
        "thread_id": str(uuid4()),
        "user": "alice",
        }}

### 6. Testing the Calendar Agent

Sample prompts:
- "Dammi le informazioni sui miei impegni per il 3 ottobre 2025."
- "Vorrei aggiungere una riunione con il team venerdì prossimo alle 15:00." -> If an event is already there:
- "Cancella l'evento" → expect confirmation first -> "Si"
- "Aggiungi un meeting domani" → expect follow-up request for missing hour/title

In [8]:
from utils.stream import print_stream

user_task = "Dammi le informazioni sui miei impegni per il 3 ottobre 2025."
inputs = {"messages": [("user", user_task)]}

out = print_stream(calendar_agent.stream(inputs, calendar_agent_config, stream_mode="values"))

================================ Human Message =================================

Dammi le informazioni sui miei impegni per il 3 ottobre 2025.
================================== Ai Message ==================================

Ecco i tuoi impegni per il **ven 03/10/2025**:

| Ora   | Titolo                   |
| ----- | ------------------------ |
| 09:30 | Standup di prodotto      |
| 11:00 | OKR review               |
| 15:00 | Riunione con il team     |


## 2. Desk Agent

The Desk Agent manages the reservation lifecycle: request, release, inspect state, and retrieve descriptive metadata. It operates on potentially contested resources (same desk, same date), so it emphasizes:
- Pre‑action validation
- Actionable failure messages (suggest next step)
- Minimal questioning (only desk ID or date if absent)

Isolation avoids leakage of calendar-specific formatting or confirmation patterns that are not relevant here.

### 1. Tool Definition

We define the tools for the Desk domain. Each returns structured data; the agent turns it into concise natural language.

#### Tools Provided
1. `get_current_date` – baseline for relative references.
2. `convert_weekday_to_date` – normalize weekday names.
3. `retrieve_desk_general_info` – metadata (location, description, allowed users).
4. `reserve_desk` – reservation (conflict & permission checks inside tool).
5. `release_desk` – release if the booking belongs to the user.
6. `get_all_reserved_desks` – list active/future reservations.


In [9]:
from tools.desk_manager import get_current_date, convert_weekday_to_date, retrieve_desk_general_info, reserve_desk, release_desk, get_all_reserved_desks

desk_tools = [
    get_current_date,
    convert_weekday_to_date,
    retrieve_desk_general_info,
    reserve_desk,
    release_desk,
    get_all_reserved_desks
]

### 2. Initializing the LLM

We reuse the same base model as the Calendar Agent for stylistic consistency and predictable latency. If future complexity emerges (e.g. multi-user optimization) we can upgrade only this model to a reasoning variant.

In [10]:
from langchain.chat_models import init_chat_model

DESK_MODEL_NAME = "openai:gpt-4o-mini-2024-07-18"

desk_llm = init_chat_model(DESK_MODEL_NAME, temperature=0)

### 3. Crafting the Agent System Prompt

The Calendar Agent prompt encapsulates:
- Role + objective 
- Formatting rules 
- Safety policies 
- Disambiguation strategy 
- Few-shot examples 

This reduces cognitive overhead and nudges the model toward correct tool sequencing without verbose reasoning leakage.

In [11]:
DESK_AGENT_PROMPT = """
# 🧭 System Prompt — “Desk Agent” per gpt4o mini

> **Lingua:** rispondi sempre in **italiano**.
> **Dominio:** prenotazioni e gestione postazioni (desk).
> **Obiettivo primario:** aiutare l’utente a **prenotare, rilasciare e consultare** informazioni e prenotazioni delle postazioni, usando in modo sicuro ed efficace i tool forniti.

---

## Ruolo & Obiettivi

Sei un **assistente per la gestione dei desk**.
Capisci l’intento dell’utente, **normalizzi date e ID**, chiami i tool quando servono, e restituisci risposte **chiare, sintetiche e operative**.

---

## Strumenti disponibili

Usa **sempre** i tool, non fare affidamento a knowledge intrinseca o inventare dati.

* `get_current_date`
* `convert_weekday_to_date`
* `retrieve_desk_general_info`
* `reserve_desk`
* `release_desk`
* `get_all_reserved_desks`

> Non spiegare nel dettaglio parametri o output dei tool all’utente; usa i risultati per guidare l’azione e la risposta.

---

## Linee guida di comportamento (Best Practices)

1. **Identifica l’intento**

   * Prenotare, rilasciare, informarsi su un desk, oppure vedere prenotazioni dell’utente.

2. **Date sempre chiare**

   * Se l’utente cita un **giorno della settimana** (“lunedì”, “mercoledì prossimo”), usa `convert_weekday_to_date` per ottenere la data **YYYY-MM-DD**.
   * Quando serve “oggi”, usa `get_current_date` per evitare ambiguità.

3. **Desk ID & permessi**

   * Non inventare ID. Se l’utente non lo indica, chiedilo con una **domanda mirata**.
   * Se utile, puoi consultare `retrieve_desk_general_info` prima di prenotare (es. per luogo/descrizione/utenti ammessi).
   * Rispetta eventuali restrizioni di accesso (il tool di prenotazione le fa rispettare).

4. **Prenotazioni**

   * Per prenotare usa `reserve_desk`.
   * Se il tool segnala che **l’utente ha già una prenotazione** quel giorno, proponi di **cancellarla** (rilasciarla) prima di procedere.
   * Se il desk è già occupato o inesistente, comunica l’esito e **offri alternative** (es. “vuoi tentare con un altro ID o un’altra data?”).

5. **Rilascio**

   * Usa `release_desk` per liberare una prenotazione dell’utente sulla data/desk indicati.
   * Se il tool indica che non è prenotato o è prenotato da un altro utente, spiega l’esito e proponi il **prossimo passo** (controllo dettagli, data corretta, ecc.).

6. **Storico dell’utente**

   * Usa `get_all_reserved_desks` per mostrare all’utente le sue prenotazioni future/attive, in modo sintetico.

7. **Error handling & veridicità**

   * **Non affermare mai** di aver prenotato/rilasciato se il tool non conferma.
   * Riporta messaggi d’errore **chiari e orientati all’azione** (cosa è andato storto, cosa proviamo ora).

8. **Stile di risposta**

   * **Breve, diretto, cortese**.
   * Metti in evidenza: **data, desk ID, esito**.
   * Usa elenchi puntati quando aiuta la leggibilità.

9. **Privacy & sicurezza**

   * Non chiedere dati non necessari.
   * Non rivelare dettagli tecnici dei tool o implementazioni interne.

---

## Strategie operative

* **Disambiguazione minima e mirata:**
  Chiedi solo ciò che manca (es. “Quale desk ID desideri?” o “Confermi la data YYYY-MM-DD?”).
* **Normalizzazione coerente:**
  Mostra sempre le date come **YYYY-MM-DD** e gli ID come forniti dall’utente (es. `A3`).
* **Conferme esplicite:**
  Dopo una prenotazione/rilascio riusciti, **conferma** con: data, desk ID, utente implicito, e **prossimi passi** (es. “Vuoi aggiungere un promemoria?” — senza creare automazioni).
* **Fallback utili:**
  In caso di desk non esistente/data non disponibile, proponi:

  * alternativa di **data** (chiedi un altro giorno)
  * alternativa di **desk ID** (chiedi un altro ID).

---

## Formati di risposta consigliati

**Conferma di prenotazione riuscita**

* “✅ Prenotazione confermata: **Desk ID** in data **YYYY-MM-DD**. Vuoi fare altro?”

**Conferma di rilascio riuscito**

* “♻️ Prenotazione rilasciata: **Desk ID** in data **YYYY-MM-DD**.”

**Richiesta di disambiguazione**

* “Per procedere mi serve il **desk ID** (es. A3). Quale vuoi usare?”

**Esito negativo con step successivi**

* “❌ Il desk **ID** risulta già occupato il **YYYY-MM-DD**. Vuoi provare con **un altro desk ID** o **un’altra data**?”

---

## Esempi rapidi

**Esempio 1 — “Prenota lunedì A3”**

1. Usa `convert_weekday_to_date` su “lunedì”.
2. Chiama `reserve_desk` con la data ottenuta e l’ID `A3`.
3. Se confermato: “✅ Prenotazione confermata: Desk A3 il 2025-09-29.”
4. Se rifiutato (già occupato/non disponibile): comunica l’errore e proponi alternative.

**Esempio 2 — “Rilascia A3 domani”**

1. Ottieni oggi con `get_current_date`, calcola “domani” chiedendo all’utente la data se non chiara, oppure fai chiedere una **data precisa**.
2. Esegui `release_desk`.
3. Conferma l’esito o proponi correzioni se l’operazione non è possibile.

**Esempio 3 — “Che desk ho prenotato?”**

1. Esegui `get_all_reserved_desks`.
2. Se vuoto: “Nessuna prenotazione trovata.”
3. Se presente: elenca per righe “Data — Desk ID”.

**Esempio 4 — “Dove si trova B2?”**

1. Esegui `retrieve_desk_general_info` con `B2`.
2. Riassumi posizione/descrizione/utenti ammessi.
3. Offri di prenotare per una data specifica.

---

## Cose da **fare**

* Chiedere **solo** le informazioni mancanti (data o desk ID).
* **Usare i tool** per ogni dato dinamico (soprattutto date).
* Mostrare sempre **esito e dettagli** (data, ID).
* Proporre **prossimi passi** quando qualcosa non va.

## Cose da **evitare**

* Non fare calcoli di calendario “a mano” se puoi usare i tool.
* Non inventare ID, date o disponibilità.
* Non rivelare logiche interne, parametri o eccezioni di implementazione.
* Non dire di aver eseguito un’azione se il tool non lo conferma.

---

## SESSION START
Greet the user only once at the beginning of a new thread. The user name is {user_name}. Current date is {current_date}.
"""

### 4. Dynamic Prompt

We can define a wrapper function for the system prompt to add user-related information or other details that may be useful. 

In [12]:
def custom_prompt_desk(state: dict, config: RunnableConfig):
    template = DESK_AGENT_PROMPT
    prompt_template = PromptTemplate(
        template=template,
        input_variables=[
            "user_name", 
            "current_date"
        ],
    )

    user_name = config['configurable']['user']
    current_date = get_current_date_util()

    system_msg = SystemMessage(
        content=prompt_template.format(
            user_name=user_name, 
            current_date=current_date
        )
    )

    return [system_msg] + state["messages"]


### 5. Building the Desk Agent

Parallels the Calendar Agent:
- Tool binding
- Dynamic prompt (user + current date)
- Dedicated memory


In [13]:
desk_agent_memory = MemorySaver()

desk_agent = create_react_agent(
    model=desk_llm.bind_tools(desk_tools, parallel_tool_calls=False),
    tools=desk_tools,
    prompt=custom_prompt_desk,
    checkpointer=desk_agent_memory,
)

desk_agent_config = {
    "configurable": {
        "thread_id": str(uuid4()),
        "user": "alice",
        }}

### 6. Testing the Desk Agent

Suggested cases:
- "Quali prenotazioni ho per la prossima settimana?"
- "Mi cancelli tutte le mie prenotazioni?"
- "Aggiungi una prenotazione per il desk5" ex: "In realtà volevo prenotarlo per il 14 ottobre"
- "Mi prenoti il desk A1 per il 13 ottobre?"

Observe tone: succinct, outcome + next step.

In [15]:
from utils.stream import print_stream

user_task = "Quali prenotazioni ho per la prossima settimana?"
inputs = {"messages": [("user", user_task)]}

print_stream(desk_agent.stream(inputs, desk_agent_config, stream_mode="values"))

================================ Human Message =================================

Quali prenotazioni ho per la prossima settimana?
================================== Ai Message ==================================
Tool Calls:
  get_all_reserved_desks (call_VOPoiRWofVGHl2TMOBayoTPF)
 Call ID: call_VOPoiRWofVGHl2TMOBayoTPF
  Args:
================================= Tool Message =================================
Name: get_all_reserved_desks

Date: 2025-09-30, Desk ID: A5
Date: 2025-10-14, Desk ID: A5
================================== Ai Message ==================================

Ecco le tue prenotazioni per la prossima settimana:

- **2025-09-30** — Desk ID: **A5**
- **2025-10-14** — Desk ID: **A5**

Hai bisogno di ulteriore assistenza con queste prenotazioni?


## Multi-Agent System

We now compose the two agents into a coordinated system. Core principle: **the Central Agent does not re‑implement domain logic**; it acts purely as an intelligent router.

Adopted pattern:
- Wrap each agent as a tool (`chat_with_calendar_agent`, `chat_with_desk_agent`)
- Central Agent picks which tool to invoke based on inferred intent
- Separate memories maintained through supplied thread IDs in the config

Benefits:
- Decoupling: easy to swap/upgrade a domain agent
- Scalability: adding a new domain = new prompt + tool wrapper
- Traceability: routing logs distinct from domain action logs

Next: define the agent wrappers and the coordination prompt.

### 1. Re-instantiating Domain Agents for Multi-Agent Coordination

In this section, we re-instantiate the Calendar Agent and Desk Agent to ensure each has a fresh, isolated memory and configuration. This is necessary for multi-agent orchestration, allowing the central agent to route requests to dedicated, independent agent instances for calendar and desk management.

In [16]:
calendar_agent_memory = MemorySaver()

calendar_agent = create_react_agent(
    model=calendar_llm.bind_tools(calendar_tools, parallel_tool_calls=False),
    tools=calendar_tools,
    prompt=custom_prompt_calendar,
    checkpointer=calendar_agent_memory,
)

In [17]:
from langgraph.prebuilt import create_react_agent

from langgraph.checkpoint.memory import MemorySaver

desk_agent_memory = MemorySaver()

desk_agent = create_react_agent(
    model=desk_llm.bind_tools(desk_tools, parallel_tool_calls=False),
    tools=desk_tools,
    prompt=custom_prompt_desk,
    checkpointer=desk_agent_memory,
)

### 2. Agent As-a-Tool

In this section, we wrap each domain agent (Calendar Agent and Desk Agent) as a callable tool. This allows the Central Agent to delegate user requests to the appropriate specialized agent based on the detected intent. We expose each agent as a tool enabling modular orchestration and maintaining clear separation of responsibilities between calendar management and desk reservation functionalities. 


Each tool wrapper (e.g., `chat_with_calendar_agent`, `chat_with_desk_agent`) receives the user's input and a `RunnableConfig` object. The wrapper extracts relevant fields (like `user` and agent-specific `thread_id`) from `config['configurable']` and forwards them to the sub-agent when invoking it. This mechanism allows the central agent to route requests while preserving user identity and session continuity for each specialized agent.

**Example:**

```python
calendar_agent_config = {
    "configurable": {
        "thread_id": config['configurable']['calendar_agent_id'],
        "user": config['configurable']['user'],
    }
}
```

This pattern ensures that each agent operates with the correct user context and maintains a separate conversation history.

In [18]:
from langchain_core.tools import tool

@tool
def chat_with_calendar_agent(
    user_input: str,
    config : RunnableConfig
    ): 
    """
    Chat with the calendar management agent.

    Inputs:
    - user_input: str : The user's input message.
    - config : RunnableConfig.

    Output
    - calendar agent's response as str.
    """
    inputs = {"messages": [("user", user_input)]}
    calendar_agent_config = {
        "configurable": {
            "thread_id": config['configurable']['calendar_agent_id'],
            "user": config['configurable']['user'],
            }}
    return calendar_agent.invoke(inputs, calendar_agent_config)['messages'][-1].content

In [19]:
@tool
def chat_with_desk_agent(
    user_input: str,
    config : RunnableConfig
    ):
    """
    Chat with the desk management agent.
    
    Inputs:
    - user_input: str : The user's input message.
    - config : RunnableConfig.
    
    Output
    - desk agent's response as str.
    """
    inputs = {"messages": [("user", user_input)]}
    desk_agent_config = {
        "configurable": {
            "thread_id": config['configurable']['desk_agent_id'],
            "user": config['configurable']['user'],
            }}
    return desk_agent.invoke(inputs, desk_agent_config)['messages'][-1].content

### 3. Defining the Coordinator Agent Prompt

In this section, we define the system prompt for the Coordinator Agent, which orchestrates interactions between the Calendar Agent and Desk Agent. The prompt includes:

- **Role definition:** The Coordinator Agent acts as a router, delegating user requests to the appropriate specialized agent.
- **Delegation guidelines:** It must identify user intent (calendar vs desk), use the correct tool, and never handle domain logic directly.
- **Clarification strategy:** If the intent is ambiguous, the agent asks clarifying questions before routing.
- **Context management:** Maintains conversation context and ensures coherent multi-turn interactions.

In [20]:
CENTRAL_AGENT_PROMPT = """
# 🧭 System Prompt — "Central Agent"

You are the central coordinator for multiple specialized agents: a Calendar Agent and a Desk Agent. Your role is to understand user requests and delegate them to the appropriate agent based on the context.

Do not handle requests directly; always route them to the relevant agent using the provided tools.

Calendar Agent capabilities:
* Manage calendar events: view, add, delete.

Desk Agent capabilities:
* Manage desk reservations: view, reserve, release.

## Guidelines for Delegation
1. **Identify Intent**: Determine if the user's request pertains to calendar management or desk reservations.
2. **Use Tools**: Always use the `chat_with_calendar_agent` or `chat_with_desk_agent` tools to forward requests.
3. **Clarify When Needed**: If the request is ambiguous, ask a clarifying question to the user before delegating.
4. **Maintain Context**: Keep track of the conversation context to ensure coherent interactions across multiple turns.

## CONVERSATION START
Greet the user only once at the beginning of a new thread. The user name is {user_name}. Current date is {current_date}.
"""

### 4. Dynamic Prompt

We can define a wrapper function for the system prompt to add user-related information or other details that may be useful. 

In [21]:
def custom_prompt_modifier(
    state: dict, 
    config: RunnableConfig
    ):
    template = CENTRAL_AGENT_PROMPT
    prompt_template = PromptTemplate(
        template=template,
        input_variables=[
            "user_name", 
            "current_date"
        ],
    )

    user_name = config['configurable']['user']
    current_date = get_current_date_util()

    system_msg = SystemMessage(
        content=prompt_template.format(
            user_name=user_name, 
            current_date=current_date
        )
    )

    return [system_msg] + state["messages"]

### 5. Initializing the LLM
In this section, we initialize the language model (LLM) for the Central Agent. For this orchestration layer, we use a lightweight model (`openai:gpt-4o-mini-2024-07-18`) to ensure fast routing and low latency. The Central Agent does not perform domain reasoning; it only delegates requests to the specialized agents.


In [22]:
CENTRAL_AGENT_MODEL_NAME = "openai:gpt-4o-mini-2024-07-18"

central_agent_llm = init_chat_model(CENTRAL_AGENT_MODEL_NAME, temperature=0)

### 6. Building the Coordinator Agent


We setup the Coordinator Agent to route user requests to the appropriate specialized agent, maintaining modularity and clear separation of responsibilities.

The `config` dictionary for the Coordinator Agent must include unique thread IDs for itself and for each sub-agent (calendar and desk). This enables the central agent to pass the correct context to each specialized agent, ensuring that conversation history and user state are maintained independently for each domain. For example:

```python
central_agent_config = {
    "configurable": {
        "thread_id": str(uuid4()),           
        "user": "alice",
        "calendar_agent_id": str(uuid4()),   
        "desk_agent_id": str(uuid4())        
    }
}
```

When the central agent delegates a request, it forwards the relevant sub-agent ID in the config, so each agent preserves its own isolated memory and context. 

In [23]:
central_agent_memory = MemorySaver()

central_agent = create_react_agent(
    model=central_agent_llm.bind_tools([chat_with_calendar_agent, chat_with_desk_agent], parallel_tool_calls=False),
    tools=[chat_with_calendar_agent, chat_with_desk_agent],
    prompt=custom_prompt_modifier,
    checkpointer=central_agent_memory,
)

central_agent_config = {
    "configurable": 
    {
        "thread_id": str(uuid4()), 
        "user": "alice",
        "calendar_agent_id": str(uuid4()),
        "desk_agent_id": str(uuid4())    
    }}

### 7. Testing the Multi-Agent System

You can test the multi-agent system by asking the same single-domain questions you used for the calendar_agent and desk_agent. This time, the orchestrator (central coordinator agent) will automatically route each request to the appropriate specialized agent, ensuring correct handling and modular separation of logic.

In [24]:
from utils.stream import display_stream

user_task = "Mi dai gli impegni del 29?"
inputs = {"messages": [("user", user_task)]}


display_stream(central_agent.stream(inputs, central_agent_config, stream_mode="values"), thinking=False)

================================ Human Message =================================

Mi dai gli impegni del 29?
================================== Ai Message ==================================
Tool Calls:
  chat_with_calendar_agent (call_zhIvI7M59AumEIpZGMAL3S5e)
 Call ID: call_zhIvI7M59AumEIpZGMAL3S5e
  Args:
    user_input: Mostrami gli impegni del 29 settembre 2025.
================================= Tool Message =================================
Name: chat_with_calendar_agent

Ecco gli impegni del 29 settembre 2025:

| Ora   | Titolo                |
| ----- | --------------------- |
| 09:30 | Weekly kickoff        |
| 10:30 | 1:1 con Marco        |
| 15:00 | Revisione roadmap     |
| 16:30 | Sync marketing        |
================================== Ai Message ==================================

Ecco gli impegni del 29 settembre 2025:

| Ora   | Titolo                |
| ----- | --------------------- |
| 09:30 | Weekly kickoff        |
| 10:30 | 1:1 con Marco        |
| 15:00 | Revisi